This notebook is a sketch for the upcoming plot.ly Dashboard

In [2]:
%matplotlib inline
import json
import pandas as pd
from datetime import datetime
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [3]:
df = pd.DataFrame(json.load(open('database.json', 'r'))['logs'])
df.data = list(map(lambda x: datetime.fromtimestamp(x), df.data))
df['adicional_totalizado'] = df.adicional.apply(lambda x: sum(x))

In [4]:
period_start = min(df.data)
period_end = max(df.data)
columns = ['comp_energia','comp_tributos','comp_distribuicao', 'comp_enc.setoriais','comp_transmissao', 'adicional_totalizado']
desc = ['Energia','Tributos','Distribuição', 'Enc. Setoriais', 'Transmissão','Adicionais']
contrib = {}
total = df.loc[(df.data >= period_start) & (df.data <= period_end), 'total']
for i in columns:
    contrib[i] = np.mean(df.loc[(df.data >= period_start) & (df.data <= period_end), i]/total)
contrib = {k: v/sum(contrib.values()) for k, v in contrib.items()}

In [8]:
chart = list()
for column in zip(columns,desc):
    chart.append(go.Bar(
        x=df.data,
        y=df.loc[:,column[0]],
        name=column[1]
    ))
    
layout = go.Layout(
    barmode='stack',
    title='Tarifa mensal'
)

chart.append(go.Scatter(
    x=df.data,
    y=df.loc[:,'total'],
    name='Total',
    mode='markers',
    opacity=0
))

py.iplot(go.Figure(data=chart, layout=layout)) #adicionais não estão 100%


fig = {
  "data": [
    {
      "values": list(contrib.values()),
      "labels": [
        'Energia','Tributos','Distribuição', 'Enc. Setoriais', 'Transmissão', 'Adicionais'
      ],
      "name": "Tarifa média",
      "hoverinfo":"label+percent",
      "hole": .6,
      "type": "pie"
    }],
  "layout": {
        "title":"Tarifa média",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": 'R$'+str(round(total.mean(),2)),
            }
        ]
    }
}
py.iplot(fig)